In [5]:
    #IMPORTING DEPENDENCIES

In [6]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk

2022-11-25 22:58:38.493652: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-25 22:58:39.236255: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-25 22:58:39.236316: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-25 22:58:39.383925: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-25 22:58:40.470515: W tensorflow/stream_executor/platform/de

In [7]:
r = pd.read_csv('IMDB Dataset.csv')
r

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [8]:
#DATA PREPROCESSING

In [9]:
r['review'] = r['review'].str.replace(r'<[^<>]*>', '')
r['review'] = r['review'].replace(to_replace=r'^https?:\/\/.*[\r\n]*',value='')
r['review'] = r['review'].str.lower()

/tmp/ipykernel_16609/3475780766.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  r['review'] = r['review'].str.replace(r'<[^<>]*>', '')


In [10]:
d = pd.DataFrame({'strings': ['a#bc1!', 'a(b$c']})

d.strings.str.replace('[^a-zA-Z0-9]', '')

/tmp/ipykernel_16609/19091054.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  d.strings.str.replace('[^a-zA-Z0-9]', '')


0    abc1
1     abc
Name: strings, dtype: object

In [11]:
r

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
49995,i thought this movie did a down right good job...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,i am a catholic taught in parochial elementary...,negative
49998,i'm going to have to disagree with the previou...,negative


In [12]:
#RECURRING WORDS

In [13]:

# for i in range(len(r['review'])):
#     pd.Series(" ".join(r['review']).split()).value_counts()>100

In [14]:
rem_stopwords = ['it.', 'could', 'made', '-', 'get', 'also', 'would', 'get', 'seen', 'go', 'two', 'one']

In [15]:
#REMOVING STOPWORDS

In [16]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
r['review'] = r['review'].apply(lambda x: " ".join([word for word in x.split() if word not in stop_words]))
r['review'] = r['review'].apply(lambda x: " ".join([word for word in x.split() if word not in rem_stopwords]))

[nltk_data] Downloading package stopwords to /home/sourav/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
#TOKENIZATION

In [18]:
# from pandarallel import pandarallel

# pandarallel.initialize(progress_bar=True)
# r['tokenized'] = r['review'].parallel_apply(lambda x: nltk.word_tokenize(x))

In [19]:
# r['review'].apply(lambda x: len(x.split()))

In [20]:

from textblob import Word
r['lemmatized'] = r['review'].apply(lambda x: " ".join(Word(word).lemmatize() for word in str(x).split()))

In [21]:
recuuring_words = pd.Series(" ".join(r['review']).split()).value_counts()

In [22]:
(recuuring_words>10).value_counts()     #choosing vocab_size

False    383903
True      38418
dtype: int64

In [23]:
r

,review,sentiment,lemmatized
0,reviewers mentioned watching 1 oz episode hook...,positive,reviewer mentioned watching 1 oz episode hooke...
1,wonderful little production. filming technique...,positive,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...,positive,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...,negative,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu...",positive,"petter mattei's ""love time money"" visually stu..."
...,...,...,...
49995,thought movie right good job. creative origina...,positive,thought movie right good job. creative origina...
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative,"bad plot, bad dialogue, bad acting, idiotic di..."
49997,catholic taught parochial elementary schools n...,negative,catholic taught parochial elementary school nu...
49998,i'm going disagree previous comment side malti...,negative,i'm going disagree previous comment side malti...


In [24]:
# pos = 0
# for i in range(r.shape[0]):
#                if r.iloc[i]['sentiment'] == 'positive':
#                     pos = pos+1
# neg = r.shape[0]-pos
# posper = pos/r.shape[0]*100
# negper = neg/r.shape[0]*100
# print(posper)
# print(negper)

In [25]:
#ENCODING

In [26]:
reviews = r['lemmatized'].values
labels = r['sentiment'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)    #converting positive  to 1 and negative to 0

In [27]:
#SPLITTING TRAINING AND TESTING DATA

In [28]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify= encoded_labels)

In [29]:
#HYPERPARAMETERS OF MODEL

In [46]:
vocab_size = 10000
tok = ''
embedding_dim = 100
max_length = 300
padding_type = 'post' 
trunc_type = 'post'

In [47]:
#TOKENIZE SENTENCES

In [48]:
tokenize = Tokenizer(num_words = vocab_size, oov_token =tok)
tokenize.fit_on_texts(train_sentences)
word_index = tokenize.word_index

In [49]:
#CONVERTING TRAIN DATASET TO SEQUENCES AND PAD SEQUENCES

In [50]:
train_sequences = tokenize.texts_to_sequences(train_sentences)
train_padding = pad_sequences(train_sequences, padding = 'post', maxlen = max_length)

In [51]:
#CONVERTING TEST DATASET TO SEQUENCES AND PAD SEQUENCES

In [52]:
test_sequences = tokenize.texts_to_sequences(test_sentences)
test_padding = pad_sequences(test_sequences, padding = 'post', maxlen = max_length)

In [53]:
#BUILDING MODEL

In [54]:
#Initialisation

In [55]:
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [56]:
#COMPILING

In [57]:
model.compile(loss = 'binary_crossentropy',
              optimizer  = 'adam',
              metrics = ['accuracy'])

In [58]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 300, 100)          1000000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              84480     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 24)                3096      
                                                                 
 dense_3 (Dense)             (None, 1)                 25        
                                                                 
Total params: 1,087,601
Trainable params: 1,087,601
Non-trainable params: 0
_________________________________________________________________


In [59]:
#TRAINING

In [60]:
n_epochs= 5
history = model.fit(train_padding, train_labels,
                   epochs=n_epochs, verbose= 1,
                   validation_split = 0.1) 

Epoch 1/5


2022-11-25 23:08:28.144483: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 40500000 exceeds 10% of free system memory.


1055/1055 [==============================] - 306s 285ms/step - loss: 0.3714 - accuracy: 0.8353 - val_loss: 0.3303 - val_accuracy: 0.8635
Epoch 2/5
1055/1055 [==============================] - 293s 277ms/step - loss: 0.2423 - accuracy: 0.9079 - val_loss: 0.3373 - val_accuracy: 0.8752
Epoch 3/5
1055/1055 [==============================] - 296s 281ms/step - loss: 0.1826 - accuracy: 0.9332 - val_loss: 0.3408 - val_accuracy: 0.8517
Epoch 4/5
1055/1055 [==============================] - 285s 270ms/step - loss: 0.1464 - accuracy: 0.9487 - val_loss: 0.3528 - val_accuracy: 0.8763
Epoch 5/5
1055/1055 [==============================] - 302s 286ms/step - loss: 0.1324 - accuracy: 0.9537 - val_loss: 0.4110 - val_accuracy: 0.8755


In [62]:
prediction = model.predict(test_padding)


391/391 [==============================] - 26s 65ms/step


In [63]:
#LABELNG BASED ON PROBABILITY

In [85]:
p_labels = []
for i in prediction:
    if i > 0.5:
        p_labels.append(1)
    elif i==0.5:
        p_labels.append(2)
    else:
        p_labels.append(0)
print("Accuracy: ", accuracy_score(test_labels,p_labels))

ValueError: Found input variables with inconsistent numbers of samples: [12500, 8]

In [88]:
sentence = ["The movie was best",
           "The plot of movie was terrible",
           "The actors did a very good job",
           'beautiful movie. loved it',
           "it was boring",
           "VFX were a spectacle to see",
           "i loved the movie",
           " It stinks"]
sequences = tokenize.texts_to_sequences(sentence)
padding = pad_sequences(sequences, padding = 'post', maxlen = max_length)
prediction = model.predict(padding)
p_labels = []
for i in prediction:
    if i > 0.5:
        p_labels.append(1)
    elif i==0.5:
        p_labels.append(2)
    else:
        p_labels.append(0)
for i in range (len(sentence)):
    if p_labels[i]==1:
        s= 'Positive'
    elif p_labels[i]==2:
        s='Neutral'
    else:
        s='Negative'
    print("Sentiment"+str(i)+" is: ",s)


1/1 [==============================] - 0s 99ms/step
Sentiment0 is:  Positive
Sentiment1 is:  Negative
Sentiment2 is:  Positive
Sentiment3 is:  Positive
Sentiment4 is:  Negative
Sentiment5 is:  Positive
Sentiment6 is:  Positive
Sentiment7 is:  Negative


In [87]:
model.save('sentimental.h5')